<a href="https://colab.research.google.com/github/edanielacero/SIS-INT-2023-PRAC-3/blob/main/Proyecto_oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random
import copy

# State class representing othello board

In [60]:
class Status:
    def __init__(self, size=8):
        self.size = size
        self.board = [['.' for _ in range(size)] for _ in range(size)]
        # Initialize chips (pieces of the game)
        half = size // 2
        self.board[half-1][half-1] = self.board[half][half] = 'O'
        self.board[half][half-1] = self.board[half-1][half] = 'X'

        self.letters_column = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[:size]


    def show_board(self):
        print('  ' + ' '.join(self.letters_column))

        for i, row in enumerate(self.board):
            print(f'{i+1} ', end='')

            for box in row:
                print(box, end=' ')
            print()
        print()

    def letter_to_number_column(self, letter):
        return ord(letter) - 65  # Convert the letter to a column number (0-7)

    def make_move(self, row, column_letter, player):
      row = row - 1
      column = self.letter_to_number_column(column_letter)
      if self.board[row][column] == ".":
        self.board[row][column] = player
        opponent = 'O' if player == 'X' else 'X'
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in directions:
            i, j = row + dir_f, column + dir_c
            if not self.is_inside_of_board(i, j) or self.board[i][j] != opponent:
                continue

            i, j = i + dir_f, j + dir_c
            while self.is_inside_of_board(i, j) and self.board[i][j] == opponent:
                i, j = i + dir_f, j + dir_c

            if self.is_inside_of_board(i, j) and self.board[i][j] == player:
                while (i != row or j != column):
                    i, j = i - dir_f, j - dir_c
                    self.board[i][j] = player

    def get_valid_moves(self, player):
        valid_moves = []

        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == '.':
                    if self.is_valid_move(i, j, player):
                        row_number = i + 1
                        column_letter = chr(j + 65)  # We convert the index to letter (A-H)
                        valid_moves.append((row_number, column_letter))

        return valid_moves

    def is_valid_move(self, row, column, player):
        if self.board[row][column] != '.':
            return False

        opponent = 'O' if player == 'X' else 'X'
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in directions:
            i, j = row + dir_f, column + dir_c
            if self.is_inside_of_board(i, j) and self.board[i][j] == opponent:
                i, j = i + dir_f, j + dir_c
                while self.is_inside_of_board(i, j) and self.board[i][j] == opponent:
                    i, j = i + dir_f, j + dir_c

                if self.is_inside_of_board(i, j) and self.board[i][j] == player:
                    return True

        return False

    def is_inside_of_board(self, row, column):
        return 0 <= row < self.size and 0 <= column < self.size

    def is_game_over(self):
       return self.full_board() or (self.get_valid_moves("O")==[] and self.get_valid_moves("X")==[])

    def full_board(self):
      cont = sum(1 for row in self.board for box in row if box != ".")
      return cont == self.size * self.size

    def get_winner(self):
        x_count = sum(row.count('X') for row in self.board)
        o_count = sum(row.count('O') for row in self.board)

        if x_count > o_count:
            return 'X'
        elif o_count > x_count:
            return 'O'
        else:
            return 'Draw'

    def utility(self):
      x_count = sum(row.count('X') for row in self.board)
      o_count = sum(row.count('O') for row in self.board)
      return x_count - o_count

In [5]:
est = Status()
est.show_board()
print(est.get_valid_moves('X'))
est.make_move(3, 'D', 'X')
est.show_board()
print(est.is_game_over())
est.utility()

  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . . . . . . 
4 . . . O X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

[(3, 'D'), (4, 'C'), (5, 'F'), (6, 'E')]
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . X . . . . 
4 . . . X X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

False


3

## Terminal test

In [6]:
def terminal_test(state):
  return state.is_game_over()

## Result function

In [7]:
def result(state, action):
  succesor = copy.deepcopy(state)
  succesor.make_move(action[0][0], action[0][1], action[1])
  return succesor

## Utility function

In [8]:
def utility(state):
  return state.utility()

## Min max

In [9]:
def max_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # small value
    actions = state.get_valid_moves('X')
    print(actions)
    for action in actions:
        print("Im in MAX actions")
        successor = result(state, (action,'X')) # returns successor state
        eval_min = min_value(successor) # evaluate the state
        v = max(v, eval_min)
    return v

def min_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # big value
    actions = state.get_valid_moves('O')
    print(actions)
    for action in actions:
        print("Im in MIN actions")
        successor = result(state, (action,'O')) # returns successor state
        eval_max = max_value(successor) # evaluate the state
        v = min(v, eval_max)
    return v

def min_max(state, player):
    if player == "MAX":
        v = []
        actions = state.get_valid_moves('X')
        print()
        for action in actions:
            print("In mix max superior")
            successor = result(state, (action,'X'))
            v.append((min_value(successor), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.get_valid_moves('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor), action))
        return min(v, key=lambda x: x[0])[1]

In [10]:
initial_state = Status(4)
min_max(initial_state, "MAX")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[(3, 'D')]
Im in MIN actions
[(1, 'D'), (2, 'D')]
Im in MAX actions
[]
Im in MAX actions
[(1, 'D')]
Im in MIN actions
Im in MAX actions
[(3, 'D')]
Im in MIN actions
[(1, 'D'), (2, 'A')]
Im in MAX actions
[]
Im in MAX actions
[(1, 'D')]
Im in MIN actions
Im in MAX actions
[(2, 'D')]
Im in MIN actions
[(1, 'D')]
Im in MAX actions
[]
Im in MIN actions
[(1, 'C'), (1, 'D'), (3, 'D'), (4, 'A')]
Im in MAX actions
[(1, 'D'), (2, 'A'), (4, 'D')]
Im in MIN actions
[(4, 'A')]
Im in MAX actions
[(2, 'A'), (3, 'D'), (4, 'D')]
Im in MIN actions
[]
Im in MIN actions
[]
Im in MIN actions
[(2, 'A'), (3, 'D')]
Im in MAX actions
[(3, 'D')]
Im in MIN actions
Im in MAX actions
[]
Im in MIN actions
[(1, 'D'), (3, 'D'), (4, 'A')]
Im in MAX actions
[(4, 'A'), (4, 'D')]
Im in MIN actions
[(3, 'D')]
Im in MAX actions
[(4, 'D')]
Im in MIN actions
Im in MIN actions
[(3, 'D')]
Im in MAX actions
[(4, 'A')]
Im in MIN actions
Im in MAX actions
[(1, 'D

KeyboardInterrupt: ignored

# Min Max 𝞪-𝜷 pruning

In [11]:
def max_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # small value
    actions = state.get_valid_moves('X')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if(alpha>=beta):
          continue
        #print("Im in MAX actions")
        successor = result(state, (action,'X')) # returns successor state
        eval_min = min_value(successor, alpha, beta) # evaluate the state
        if eval_min > v:
          v = eval_min
          if v > alpha:
            alpha = v
    return v

def min_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # big value
    actions = state.get_valid_moves('O')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if alpha >= beta:
          continue
        #print("Im in MIN actions")
        successor = result(state, (action,'O')) # returns successor state
        eval_max = max_value(successor, alpha, beta) # evaluate the state
        if eval_max < v:
          v = eval_max
          if beta > v:
            beta = v
    return v

def min_max_pruning(state, player):
    alpha = float('-inf') # small value
    beta = float('inf') # big value
    if player == "MAX":
        v = []
        actions = state.get_valid_moves('X')
        print(actions)
        for action in actions:
            successor = result(state, (action,'X'))
            v.append((min_value(successor, alpha, beta), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.get_valid_moves('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor,alpha, beta), action))
        return min(v, key=lambda x: x[0])[1]

In [12]:
initial_state = Status(4)
min_max_pruning(initial_state, "MAX")

[(1, 'B'), (2, 'A'), (3, 'D'), (4, 'C')]


(1, 'B')

In [13]:
def player_vs_ai(board_size):
    status = Status(board_size)
    turn = 'X'  #  starts playing

    while not status.is_game_over():
        status.show_board()

        if turn == 'X':  # Player's Turn
            print("Its your turn.")
            valid_moves = status.get_valid_moves('X')
            if valid_moves == []:
              print("You dont have possible moves :(")
            else:
              print("Valid moves:")
              for i, (row, column) in enumerate(valid_moves):
                  print(f"{i+1}. Row: {row}, Column: {column}")

              selection = int(input("Choose your move (number): ")) - 1
              row, column = valid_moves[selection]
              status.make_move(row, column, 'X')

        else:  # AIs turn
            print("AIs turn.")
            status_copy = copy.deepcopy(status)
            move = min_max_pruning(status_copy, 'MIN')
            status.make_move(move[0], move[1], 'O')
            print("AI chose: ", move)
        turn = 'O' if turn == 'X' else 'X'

    status.show_board()
    winner = status.get_winner()

    if winner == 'Draw':
        print("It's a Draw!")
    else:
      if winner == 'O':
        print(f"¡IA wins, you lost!")
      else:
        print(f"¡You win!")

In [14]:
player_vs_ai(4)

  A B C D
1 . . . . 
2 . O X . 
3 . X O . 
4 . . . . 

Its your turn.
Valid moves:
1. Row: 1, Column: B
2. Row: 2, Column: A
3. Row: 3, Column: D
4. Row: 4, Column: C


KeyboardInterrupt: ignored

# Min max heurisitco

## Max heigth

In [40]:
max_heigth = 5

## Counting chips heuristic

In [84]:
def heuristic(status, player): # counting chips heuristic
    if player == 'X':
      ai_player = 'O'
    else:
      ai_player = 'X'
    count_player = sum(fila.count(player) for fila in status.board)
    count_ai_player = sum(fila.count(ai_player) for fila in status.board)
    return count_ai_player - count_player

## Calculate stability heuristic

In [17]:
def calculate_stability(state, player):
    table_size = len(state.board)
    corners = [(0, 0), (0, table_size-1), (table_size-1, 0), (table_size-1, table_size-1)]
    stability = 0

    for row, column in corners:
        if state.board[row][column] == player:
            stability += 1

    borders = [(0, j) for j in range(1, table_size-1)] + [(i, 0) for i in range(1, table_size-1)] + [(table_size-1, j) for j in range(1, table_size-1)] + [(i, table_size-1) for i in range(1, table_size-1)]

    for row, column in borders:
        if state.board[row][column] == player:
            stability += 0.5

    return stability

In [18]:
def calculate_mobility(state, player):
    return len(state.get_valid_moves(player))

In [19]:
est = Status(4)
est.show_board()
print(est.get_valid_moves('X'))
est.make_move(1, 'B', "X")
est.show_board()
print(calculate_stability(est, "X"))
print(calculate_stability(est, "O"))
print(est.get_valid_moves('O'))
est.make_move(1, 'A', "O")
est.show_board()
print(calculate_stability(est, "X"))
print(calculate_stability(est, "O")) # Stability 1 when there is a checker in the corner

  A B C D
1 . . . . 
2 . O X . 
3 . X O . 
4 . . . . 

[(1, 'B'), (2, 'A'), (3, 'D'), (4, 'C')]
  A B C D
1 . X . . 
2 . X X . 
3 . X O . 
4 . . . . 

0.5
0
[(1, 'A'), (1, 'C'), (3, 'A')]
  A B C D
1 O X . . 
2 . O X . 
3 . X O . 
4 . . . . 

0.5
1
0


In [52]:
# def heuristic(state, player): # seems the best one
#   return calculate_stability(state, player) + diff_chips(state, player)

In [22]:
def max_value(state, alpha, beta, h):
    #print("I have h: ", h)
    if terminal_test(state) or h == max_heigth:
        #print("ive entered hereeeeeee max")
        return heuristic(state, 'X')

    v = float('-inf') # small value
    actions = state.get_valid_moves('X')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if(alpha>=beta):
          continue
        #print("Im in MAX actions")
        successor = result(state, (action,'X')) # returns successor state
        eval_min = min_value(successor, alpha, beta, h+1) # evaluate the state
        if eval_min > v:
          v = eval_min
          if v > alpha:
            alpha = v
    return v

def min_value(state, alpha, beta, h):
    #print("I have h: ", h)
    if terminal_test(state) or h == max_heigth:
        #print("ive entered hereeeeeee min")
        return heuristic(state, 'O')

    v = float('inf') # big value
    actions = state.get_valid_moves('O')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if alpha >= beta:
          continue
        #print("Im in MIN actions")
        successor = result(state, (action,'O')) # returns successor state
        eval_max = max_value(successor, alpha, beta, h+1) # evaluate the state
        if eval_max < v:
          v = eval_max
          if beta > v:
            beta = v
    return v

def min_max_heuristic(state, player, h):
    alpha = float('-inf') # small value
    beta = float('inf') # big value
    if player == "MAX":
        v = []
        actions = state.get_valid_moves('X')
        print(actions)
        for action in actions:
            successor = result(state, (action,'X'))
            v.append((min_value(successor, alpha, beta, h), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.get_valid_moves('O')
        print(actions)
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor,alpha, beta, h), action))
        return min(v, key=lambda x: x[0])[1]

In [23]:
initial_state = Status(8)
min_max_heuristic(initial_state, "MIN", 0)

[(3, 'E'), (4, 'F'), (5, 'C'), (6, 'D')]


(3, 'E')

In [82]:
import time
import copy

def player_vs_ai_h(board_size):
    status = Status(board_size)

    print("Do you want to play as X or O? x/o")
    player_choice = input()
    if player_choice == "x":
        player = 'X'
        ai_player = 'O'
    else:
        player = 'O'
        ai_player = 'X'

    turn = 'X'  # Player starts playing

    # Initialize a list to store response times of the AI
    ai_response_times = []

    while not status.is_game_over():
        status.show_board()

        if turn == player:  # Player's turn
            print("It's your turn.")
            valid_moves = status.get_valid_moves(player)
            if not valid_moves:
                print("You don't have any possible moves :(")
            else:
                print("Valid moves:")
                for i, (row, column) in enumerate(valid_moves):
                    print(f"{i+1}. Row: {row}, Column: {column}")

                while True:
                    try:
                        selection = int(input("Choose your move (number): ")) - 1
                        if 0 <= selection < len(valid_moves):
                            row, column = valid_moves[selection]
                            status.make_move(row, column, player)
                            break
                        else:
                            print("Invalid input. Please enter a valid move.")
                    except ValueError:
                        print("Invalid input. Please enter a number.")

        else:  # AI's turn
            print("AI's turn.")
            valid_moves_ai = status.get_valid_moves(ai_player)
            if not valid_moves_ai:  # Corrected the variable name here
                print("AI doesn't have any moves")
            else:
                status_copy = copy.deepcopy(status)
                start_time = time.time()  # Register start time
                if ai_player == 'O':
                  move = min_max_heuristic(status_copy,  'MIN', 0)
                else:
                  move = min_max_heuristic(status_copy,  'MAX', 0)
                end_time = time.time()  # Register end time
                ai_response_times.append(end_time - start_time)  # Calculate response time
                status.make_move(move[0], move[1], ai_player)
                print("AI chose:", move)

        turn = ai_player if turn == player else player

    status.show_board()
    winner = status.get_winner()

    if winner == 'Draw':
        print("It's a Draw!")
    else:
        if winner == ai_player:
            print("AI wins, you lost!")
        else:
            print("You win!")

    # Count the number of discs for each player
    count_player = sum(fila.count(player) for fila in status.board)
    count_ai_player = sum(fila.count(ai_player) for fila in status.board)
    print(f"Number of {player} discs: {count_player}")
    print(f"Number of {ai_player} discs: {count_ai_player}")

    # Calculate and print the average response time for the AI
    if ai_response_times:
        avg_response_time = sum(ai_response_times) / len(ai_response_times)
        print(f"Average AI response time: {avg_response_time:.2f} seconds")
    else:
        print("AI did not make any moves.")


In [86]:
player_vs_ai_h(8)

Do you want to play as X or O? x/o
x
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . . . . . . 
4 . . . O X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

It's your turn.
Valid moves:
1. Row: 3, Column: D
2. Row: 4, Column: C
3. Row: 5, Column: F
4. Row: 6, Column: E
Choose your move (number): 1
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . X . . . . 
4 . . . X X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

AI's turn.
[(3, 'C'), (3, 'E'), (5, 'C')]
AI chose: (3, 'C')
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . O X . . . . 
4 . . . O X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

It's your turn.
Valid moves:
1. Row: 3, Column: B
2. Row: 4, Column: C
3. Row: 5, Column: F
4. Row: 6, Column: E
Choose your move (number): 2
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . O X . . . . 
4 . . X X X . . . 
5 . . . X O